In [ ]:
import csv
import itertools
import pathlib
import shutil
from math import ceil, floor, sqrt
from pathlib import Path
from typing import Callable, List, Optional, Tuple

import cv2
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import PIL
import rootutils
import torch
import torch.nn.functional as F
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from ipywidgets import Video
from scipy.stats import norm
from skimage.metrics import structural_similarity
from sklearn.model_selection import GroupShuffleSplit
from torch import Tensor
from torch.utils.data import ConcatDataset, Dataset
from torchvision.io import read_image
from tqdm.notebook import tqdm

# from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('png')

root = rootutils.setup_root(search_from=".", indicator=".project-root", pythonpath=True)

from src.data.components.dataset import (
    FetalBrainPlanesDataset,
    USVideosDataset,
    USVideosFrameDataset,
    VideoQualityDataset,
    VideosFrameDataset,
)
from src.data.components.transforms import Affine, HorizontalFlip, VerticalFlip
from src.data.utils.utils import show_numpy_images, show_pytorch_images
from src.models.fetal_module import FetalLitModule
from src.models.quality_module import QualityLitModule

path = root / "data" / "US_VIDEOS_tran_0250_playful-haze-2111"
root

In [ ]:
mu = 0
variance = 1
sigma = sqrt(variance)
x = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 100)
plt.plot(x, norm.pdf(x, mu, sigma))
plt.show()

In [ ]:
mu = 0  # mean
stdev = 0.5  # standard deviation

# `x` span of +/- 4 standard deviations
x_start = mu - 4 * stdev
x_end = mu + 4 * stdev
step = 0.01

# Calculate `x` and `y`
x = np.arange(start=x_start, stop=x_end + step, step=step)
y = norm.pdf(x=x, loc=mu, scale=stdev)
y2 = norm.pdf(x=x, loc=mu + 1, scale=stdev)


def print_v(v):
    idxs = [i for i, j in enumerate(x) if (j > v - 0.1 and j < v + 0.1)]
    idx = idxs[floor(len(idxs) / 2)]
    print(f"{v}: {y[idx]}")
    print(f"{v}: {y2[idx]}")


print_v(0.0)
print_v(0.5)

plt.plot(x, y)
plt.plot(x, y2)
plt.show()

In [ ]:
x_start = np.pi
x_end = x_start + np.pi * 2
step = 0.01

x = np.arange(start=x_start, stop=x_end + step, step=step)

y = np.cos(x)

plt.plot(x, y)
plt.show()

In [ ]:
step = 0.01


def curve(x, mean):
    cos_x_start = np.pi
    cos_x_end = x_start + np.pi * 2

    cos_x = np.arange(start=cos_x_start, stop=cos_x_end + step, step=step)
    cos_y = (np.cos(cos_x) + 1) / 2

    cos_mean_idx = floor(len(cos_y) / 2)
    # print(f"idx {cos_mean_idx}: {cos_y[cos_mean_idx]}")

    idxs = [i for i, j in enumerate(x) if (j > mean - 0.1 and j < mean + 0.1)]
    x_mean_idx = idxs[floor(len(idxs) / 2)]
    # print(f"mean {mean}: {x[x_mean_idx]}, idx {x_mean_idx}")

    if cos_mean_idx > x_mean_idx:
        cos_y = cos_y[cos_mean_idx - x_mean_idx :]
    else:
        cos_y = np.pad(cos_y, (x_mean_idx - cos_mean_idx, 0))

    if len(cos_y) > len(x):
        cos_y = cos_y[: len(x)]
    else:
        cos_y = np.pad(cos_y, (0, len(x) - len(cos_y)))

    return cos_y


x = np.arange(start=0, stop=np.pi * 6, step=step)

y_tv = curve(x, mean=np.pi * 2)
y_tt = curve(x, mean=np.pi * 3)
y_tc = curve(x, mean=np.pi * 4)
y_ot = curve(x, mean=np.pi * 1) + curve(x, mean=np.pi * 5)

y_ot_tv = y_ot + y_tv
y_tv_tt = y_tv + y_tt
y_tt_tc = y_tt + y_tc
y_ot_tc = y_ot + y_tc

fig, axes = plt.subplots(ncols=1, nrows=5, tight_layout=True, figsize=(10, 10))

axes[0].plot(x, y_tv, color="tab:blue")
axes[0].plot(x, y_tt, color="tab:orange")
axes[0].plot(x, y_tc, color="tab:green")
axes[0].plot(x, y_ot, color="tab:red")
axes[0].xaxis.set_major_locator(ticker.NullLocator())
axes[0].yaxis.set_major_locator(ticker.NullLocator())

axes[1].plot(x, y_tt, color="tab:orange")
axes[1].plot(x, y_tc, color="tab:green")
axes[1].plot(x, y_ot_tv, color="tab:red")
axes[1].xaxis.set_major_locator(ticker.NullLocator())
axes[1].yaxis.set_major_locator(ticker.NullLocator())

axes[2].plot(x, y_tv_tt, color="tab:purple")
axes[2].plot(x, y_tc, color="tab:green")
axes[2].plot(x, y_ot, color="tab:red")
axes[2].xaxis.set_major_locator(ticker.NullLocator())
axes[2].yaxis.set_major_locator(ticker.NullLocator())

axes[3].plot(x, y_tv, color="tab:blue")
axes[3].plot(x, y_tt_tc, color="tab:olive")
axes[3].plot(x, y_ot, color="tab:red")
axes[3].xaxis.set_major_locator(ticker.NullLocator())
axes[3].yaxis.set_major_locator(ticker.NullLocator())

axes[4].plot(x, y_tv, color="tab:blue")
axes[4].plot(x, y_tt, color="tab:orange")
axes[4].plot(x, y_ot_tc, color="tab:red")
axes[4].xaxis.set_major_locator(ticker.NullLocator())
axes[4].yaxis.set_major_locator(ticker.NullLocator())

fig.show()

In [ ]:
y_tv_s = y_tv * 2
y_tt_s = y_tt * 3
y_tc_s = y_tc * 2
y_ot_s = y_ot * 1 + y_ot_tv + y_ot_tc

fig, axes = plt.subplots(ncols=1, nrows=2, tight_layout=True, figsize=(10, 5))

axes[0].plot(x, y_tv, color="tab:blue")
axes[0].plot(x, y_tt, color="tab:orange")
axes[0].plot(x, y_tc, color="tab:green")
axes[0].plot(x, y_ot, color="tab:red")
axes[0].xaxis.set_major_locator(ticker.NullLocator())
axes[0].yaxis.set_major_locator(ticker.NullLocator())

axes[1].plot(x, y_tv_s, color="tab:blue")
axes[1].plot(x, y_tt_s, color="tab:orange")
axes[1].plot(x, y_tc_s, color="tab:green")
axes[1].plot(x, y_ot_s, color="tab:red")
axes[1].xaxis.set_major_locator(ticker.NullLocator())
axes[1].yaxis.set_major_locator(ticker.NullLocator())

fig.show()

In [ ]:
y_tv_s = y_tv * 3
y_tt_s = y_tt * 3
y_tc_s = y_tc * 3
y_ot_s = y_ot * 3 + y_ot_tv + y_ot_tc

y_ot_ss = y_ot * 3 + y_ot_tv + y_ot_tc + y_tv_tt + y_tt_tc

# y_tv_tt = y_tv + y_tt
# y_tt_tc = y_tt + y_tc

fig, axes = plt.subplots(ncols=1, nrows=4, tight_layout=True, figsize=(10, 5))

axes[0].plot(x, y_tv, color="tab:blue")
axes[0].plot(x, y_tt, color="tab:orange")
axes[0].plot(x, y_tc, color="tab:green")
axes[0].plot(x, y_ot, color="tab:red")
axes[0].xaxis.set_major_locator(ticker.NullLocator())
axes[0].yaxis.set_major_locator(ticker.NullLocator())

axes[1].plot(x, y_tv_s, color="tab:blue")
axes[1].plot(x, y_tt_s, color="tab:orange")
axes[1].plot(x, y_tc_s, color="tab:green")
axes[1].plot(x, y_ot_ss, color="tab:red")
axes[1].xaxis.set_major_locator(ticker.NullLocator())
axes[1].yaxis.set_major_locator(ticker.NullLocator())

axes[2].plot(x, y_tv_s, color="tab:blue")
axes[2].plot(x, y_tt_s, color="tab:orange")
axes[2].plot(x, y_tc_s, color="tab:green")
axes[2].plot(x, y_ot_s, color="tab:red")
axes[2].plot(x, y_tv_tt, color="tab:purple")
axes[2].plot(x, y_tt_tc, color="tab:olive")
axes[2].xaxis.set_major_locator(ticker.NullLocator())
axes[2].yaxis.set_major_locator(ticker.NullLocator())

axes[3].plot(x, y_tv_s, color="tab:blue")
axes[3].plot(x, y_tt_s, color="tab:orange")
axes[3].plot(x, y_tc_s, color="tab:green")
axes[3].plot(x, y_ot_s, color="tab:red")
axes[3].plot(x, y_tv_tt + y_tt_tc, color="tab:purple")
axes[3].xaxis.set_major_locator(ticker.NullLocator())
axes[3].yaxis.set_major_locator(ticker.NullLocator())

fig.show()